In [ ]:
# %pip install plotly matplotlib seaborn pandas numpy dash
# %pip install reportlab pandas matplotlib seaborn plotly

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.platypus import SimpleDocTemplate, Image, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet

# Verifica se o arquivo de dados existe antes de carregá-lo
file_path = 'Vendas.xlsx'
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Arquivo '{file_path}' não encontrado.")

# Importação da base de dados
df = pd.read_excel(file_path)

# Processamento dos dados
df_lojas_produtos = df.groupby(['ID Loja', 'Produto']).agg({
    'Quantidade': 'sum',
    'Valor Unitário': 'mean',
    'Valor Final': 'sum'
}).reset_index()

# Diretório para salvar os gráficos
output_dir = "graficos"
os.makedirs(output_dir, exist_ok=True)

# Gerar e salvar gráficos Matplotlib
plt.figure(figsize=(10, 5))
sns.barplot(data=df_lojas_produtos, x='ID Loja', y='Valor Final', hue='Produto')
plt.title('VALOR dos produtos vendidos por loja - Dezembro/2019')
plt.xticks(rotation=90)
plt.savefig(f"{output_dir}/grafico_valor.png", bbox_inches='tight')
plt.close()

plt.figure(figsize=(10, 5))
sns.barplot(data=df_lojas_produtos, x='ID Loja', y='Quantidade', hue='Produto')
plt.title('QUANTIDADE de produtos vendidos por loja - Dezembro/2019')
plt.xticks(rotation=90)
plt.savefig(f"{output_dir}/grafico_quantidade.png", bbox_inches='tight')
plt.close()

plt.figure(figsize=(10, 5))
sns.scatterplot(data=df_lojas_produtos, x='Quantidade', y='Valor Final', hue='Produto', size='Valor Unitário', sizes=(10, 500))
plt.title('QUANTIDADE x VALOR dos produtos vendidos - Dezembro/2019')
plt.xticks(rotation=90)
plt.savefig(f"{output_dir}/grafico_relacao.png", bbox_inches='tight')
plt.close()

# Criar o relatório PDF
pdf_file = "Relatorio_Vendas.pdf"
doc = SimpleDocTemplate(pdf_file, pagesize=letter)
styles = getSampleStyleSheet()
elements = []

# Adicionar título e descrição
elements.append(Paragraph("Relatório de Vendas", styles["Title"]))
elements.append(Spacer(1, 12))
elements.append(Paragraph("Este relatório apresenta a análise das vendas realizadas no mês de dezembro de 2019.", styles["Normal"]))
elements.append(Spacer(1, 12))

# Adicionar gráficos ao PDF
for img_name, title in zip(
    ["grafico_valor.png", "grafico_quantidade.png", "grafico_relacao.png"],
    ["Valor dos produtos vendidos por loja", "Quantidade de produtos vendidos por loja", "Relação entre Quantidade e Valor"]
):
    elements.append(Paragraph(title, styles["Heading2"]))
    elements.append(Image(f"{output_dir}/{img_name}", width=500, height=300))
    elements.append(Spacer(1, 12))

# Criar e salvar o PDF
doc.build(elements)
print(f"Relatório gerado com sucesso: {pdf_file}")


Relatório gerado com sucesso: Relatorio_Vendas.pdf
